In [1]:
import arcpy
from arcpy import env
import os
from statistics import mean
import numpy as np
import math
from numba import jit
import matplotlib.pyplot as plt
from arcpy.ia import *
arcpy.env.workspace = r"C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\PopGINIGIS.gdb"
arcpy.env.overwriteOutput = True

aprx = arcpy.mp.ArcGISProject(r"C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\PopGINIGIS.aprx")
for x in aprx.listMaps():
    print(x.name)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Mapstuff


In [2]:
def gini(array):
    array = np.cumsum(array) #Cumulative Sum array
    equality = (len(array)*array[-1])/2 #Getting area of equality triangle
    lorenze = sum(array) #Getting area of lorenze curve
    
    return ((equality - lorenze)/equality) #Final result

#Removed Aleutian Islands in shapefile because of American exceptionalism; the mask was too big to process

countryborders = r'C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\PopGINIGIS.gdb\worldcountries_aleutiansevil'
countrymask = r'C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\PopGINIGIS.gdb\NEW_SELECTION'
worldmap = r'C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\GHS_POP_E2020_GLOBE_R2023A_4326_30ss_V1_0.tif'
worldmap = Divide(worldmap, 1000) #Dividing value of pixels by 1000 because some are 'too big'


giniarray = []
with arcpy.da.SearchCursor(countryborders, ["SHAPE@","COUNTRY"]) as cursor: #Looping through each row in the table
    for row in cursor:
        print(row[1])
        extension = row[0].extent #Getting extent values to be used in clip
        print(extension.XMin, extension.XMax, '\n', extension.YMin, extension.YMax, '\n ---------------')
        

        country = row[1]
        countrynospace = country.replace(' ', '_')
        countrynodash = countrynospace.replace('-', '_') #Formatting state names to be readable

        select = "COUNTRY = '{}'".format(country)
        print(select)
        arcpy.Select_analysis(countryborders, 'NEW_SELECTION', select) #Selecting row


        out_raster = arcpy.management.Clip(
            in_raster=r'C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\GHS_POP_E2020_GLOBE_R2023A_4326_30ss_V1_0.tif',
            rectangle= str(extension.XMin) + " " + str(extension.YMin) + " " + str(extension.XMax) + " " + str(extension.YMax),
            out_raster=r"C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\PopGINIGIS.gdb\mask_{}".format(countrynodash),
            in_template_dataset=countrymask,
            nodata_value="1.79e+308",
            clipping_geometry="ClippingGeometry",
            maintain_clipping_extent="MAINTAIN_EXTENT"
        ) #Extracting mask of state population distribution

        #Converting to array to be analyzed, nans set to specific number to be filtered out
        arr = arcpy.RasterToNumPyArray(r'C:\Users\nicks\OneDrive\Desktop\VSprojects\PopGINI\PopGINIGIS\PopGINIGIS.gdb\mask_{}\Band_1'.format(countrynodash),nodata_to_value= .4206942069)
        
        print("Conversion Done")
        
        #Filtering
        nums = []
        for array in arr:
            for num in array:
                if num != .4206942069:
                    nums.append(num)

        print("Array Done")

        numssort = sorted(nums) #Sort

        print("Sorted")

        genie = gini(numssort) #Calculating Gini

        print("Genied:" + str(genie))

        giniarray.append(genie) #Adding to POPGini array to be appended
        

Afghanistan
60.504163319381746 74.91573610483744 
 29.406109078271413 38.471982115882014 
 ---------------
COUNTRY = 'Afghanistan'
Conversion Done
Array Done
Sorted
Genied:0.9873819522070021
Albania
19.288536139731093 21.053327082678752 
 39.64500013185068 42.66034504838535 
 ---------------
COUNTRY = 'Albania'
Conversion Done
Array Done
Sorted
Genied:0.9303224495279041
Algeria
-8.667217736109592 11.986473194982008 
 18.97639109017672 37.08985405522053 
 ---------------
COUNTRY = 'Algeria'
Conversion Done
Array Done
Sorted
Genied:0.992670356048715
American Samoa
-170.82322700662104 -170.56187306179234 
 -14.37555495179953 -14.254308877852374 
 ---------------
COUNTRY = 'American Samoa'
Conversion Done
Array Done
Sorted
Genied:0.7295994525388034
Andorra
1.4213911581754815 1.7817181130005866 
 42.43638210295177 42.655964162016545 
 ---------------
COUNTRY = 'Andorra'
Conversion Done
Array Done
Sorted
Genied:0.922004571234206
Angola
11.731245081195596 24.08444506468804 
 -18.0163910129348

C:\Users\nicks\AppData\Local\Temp\ipykernel_23664\1757743209.py:6: RuntimeWarning: invalid value encountered in scalar divide
  return ((equality - lorenze)/equality)


Genied:nan
Antigua and Barbuda
-61.89110879533462 -61.666946025039 
 16.98971809271586 17.72430004301263 
 ---------------
COUNTRY = 'Antigua and Barbuda'
Conversion Done
Array Done
Sorted
Genied:0.8490741619046253
Argentina
-73.58229986214889 -53.650008829244044 
 -55.05167305336869 -21.78051782863485 
 ---------------
COUNTRY = 'Argentina'
Conversion Done
Array Done
Sorted
Genied:0.9942002058761109
Armenia
43.45416337414048 46.62053610485743 
 38.84114510496266 41.29705397416162 
 ---------------
COUNTRY = 'Armenia'
Conversion Done
Array Done
Sorted
Genied:0.9678420691138788
Aruba
-70.05966389097574 -69.87486388006397 
 12.41110900287822 12.62777315400789 
 ---------------
COUNTRY = 'Aruba'
Conversion Done
Array Done
Sorted
Genied:0.654113696663266
Australia
112.90720923371919 158.9603723828792 
 -54.753891022252844 -10.135690924096835 
 ---------------
COUNTRY = 'Australia'
Conversion Done
Array Done
Sorted
Genied:0.9973158283663793
Austria
9.533573262196342 17.166382250940178 
 46.

In [3]:
countryborders = arcpy.AddField_management(countryborders, 'PopGINI', "FLOAT") #Creating new field

In [4]:
#Adding values to PopGini array
cursor = arcpy.da.UpdateCursor(countryborders, "PopGINI") 
for row, value in zip(cursor, giniarray):
    print(row)
    row[0] = value
    print(row)
    cursor.updateRow(row)

[0.9873819351196289]
[0.9873819522070021]
[0.9303224682807922]
[0.9303224495279041]
[0.9926703572273254]
[0.992670356048715]
[0.7295994758605957]
[0.7295994525388034]
[0.9220045804977417]
[0.922004571234206]
[0.9879578351974487]
[0.9879578419294878]
[0.5946823358535767]
[0.5946823426891267]
[None]
[nan]
[0.8490741848945618]
[0.8490741619046253]
[0.9942002296447754]
[0.9942002058761109]
[0.9678420424461365]
[0.9678420691138788]
[0.6541137099266052]
[0.654113696663266]
[0.9973158240318298]
[0.9973158283663793]
[0.8852834105491638]
[0.8852834270804755]
[0.9784474968910217]
[0.9784474786549523]
[0.8947671055793762]
[0.8947671257704922]
[0.9869222044944763]
[0.9869222013240448]
[0.8381708264350891]
[0.8381708080962972]
[0.6832587122917175]
[0.6832587349659724]
[0.6481463313102722]
[0.6481463209316568]
[0.9675001502037048]
[0.9675001529219168]
[0.7797373533248901]
[0.7797373805663608]
[0.977344274520874]
[0.9773442794886743]
[0.9420948028564453]
[0.9420947849433209]
[0.4996556043624878]
[0.4